## Investigates the gains for different entropy-coding schemes

In [1]:
import bench_utils
from algos.alg_fpd_extended import FpdExtended
from shapely.geometry import shape
from algos.fpd_extended_lib.compress import get_zz_encoded_delta
from algos.fpd_extended_lib.decompress import decode_header
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
fpde = FpdExtended()

In [2]:
# Load a bunch of data-sets
# chunks[delta_size] - contains a list of chunks, first indexed by the number of bits used per delta


from collections import defaultdict
from bitarray import bitarray
from algos.fpd_extended_lib import cfg

dt_to_coord = defaultdict()
dt_freq = defaultdict(lambda: defaultdict(int))
dt_xs = []
dt_ys = []
chunks = defaultdict(list)

def append_deltas(dataset):
    df, idxs = bench_utils.read_dataset(dataset)
    for idx in idxs:
        bin = fpde.compress(shape(df.iloc[idx]))[1]

        cfg.offset = 0
        delta_size, type = decode_header(bin)
        chks = fpde.get_chunks(bin, include_ring_start=False)[0]
        # Go though deltas
        for chk in chks:
            chk_dts = []
            for i in range(len(chk) - 1):
                dt_x = get_zz_encoded_delta(chk[i][0], chk[i + 1][0])
                dt_y = get_zz_encoded_delta(chk[i][1], chk[i + 1][1])
                dt_to_coord[dt_x] = chk[i + 1][0] - chk[i][0]
                dt_to_coord[dt_y] = chk[i + 1][1] - chk[i][1]
                dt_freq[delta_size][dt_x] += 1
                dt_freq[delta_size][dt_y] += 1
                dt_xs.append(dt_x)
                dt_ys.append(dt_y)
                chk_dts += [dt_x, dt_y]
            chunks[delta_size].append(chk_dts)

                


append_deltas("data/lund_building_highway.json")
append_deltas("data/world_7_dec.json")
append_deltas("data/sweden.json")

In [3]:
dt_freq_sorted = defaultdict(lambda: defaultdict(int))
for bit_size in sorted(dt_freq.keys()):
    freqs = sorted(dt_freq[bit_size].items(), key=lambda x: -x[1])
    dt_freq_sorted[bit_size] = freqs

print(dt_freq_sorted[12])

[(0, 2200), (857, 438), (858, 427), (54, 423), (322, 413), (53, 405), (25, 402), (108, 401), (1179, 387), (643, 378), (5, 372), (214, 370), (644, 369), (321, 367), (27, 356), (107, 354), (26, 352), (52, 352), (28, 350), (51, 350), (1180, 349), (105, 345), (13, 344), (6, 343), (3, 341), (213, 341), (536, 340), (16, 339), (535, 338), (9, 337), (10, 332), (8, 328), (17, 327), (1502, 325), (590, 324), (430, 323), (1716, 323), (966, 319), (589, 319), (7, 315), (15, 315), (106, 314), (4, 311), (21, 310), (11, 308), (14, 308), (55, 306), (429, 305), (23, 304), (12, 303), (1287, 303), (965, 301), (1501, 301), (29, 300), (750, 299), (697, 299), (267, 298), (24, 297), (31, 295), (79, 293), (749, 293), (35, 293), (1288, 292), (18, 292), (19, 290), (268, 290), (1715, 287), (160, 284), (911, 283), (50, 282), (45, 282), (162, 282), (48, 281), (2146, 280), (1, 279), (30, 275), (47, 274), (22, 274), (751, 273), (49, 273), (1071, 272), (912, 271), (56, 270), (209, 269), (2, 269), (376, 269), (80, 269),

In [4]:
import math
from intersection.plotting import xs, ys
import numpy as np

#sns.kdeplot()




HEIGHT = math.ceil(len(dt_freq_sorted.keys()) / 3)
WIDTH = 4

# #Plot delta distribution for all 
plt.figure(figsize=(HEIGHT * 3, HEIGHT))

fig, axes = plt.subplots(HEIGHT, WIDTH)

for idx, bit_size in enumerate(sorted(dt_freq.keys())):
    chks = chunks[bit_size]
    dts = [item for sublist in chks for item in sublist]

    curr_height, curr_width = idx // WIDTH, idx % WIDTH
    current_axis = axes[curr_height, curr_width]
    current_axis.set_title(f"Delta size: {bit_size}, {len(dts)}", pad=15, fontsize=10)

    ax = sns.kdeplot(dts, ax=current_axis)
    current_axis.set_xlim(0, max(dts))
    current_axis.set(ylabel=None)

fig.supylabel('Density')
fig.supxlabel('Delta values')
fig.suptitle('Distribution of delta values for geometries with different delta bit lengths', y=1.02)
fig.set_figwidth(HEIGHT * 3)
fig.set_figheight(HEIGHT * 3)

# plt.tight_layout()

plt.show()

In [ ]:
# for i in range(1):
#     idx = random.randint(0, len(deltas_fp))
#     print("---")
#     print("Delta in FP and LONG:    ", f'{deltas_fp[idx]:.14f}', '        ', deltas_long[idx])
#     print("Converting to f32:       ", f'{struct.unpack("!f", struct.pack("!f", deltas_fp[idx]))[0]:.14f}')
#     print("Required bits:           ", math.ceil(math.log2(deltas_long[idx] + 1)))
#     bin = util.int2ba(deltas_long[idx], length=64)
#     print("ZZ-encoded:              ", end=None)
#     util.pprint(bin, width=200)

#     bin = bitarray()
#     print("FP directly to bin:      ", end=None)
#     bin.frombytes(struct.pack("!d", deltas_fp[idx]))
#     util.pprint(bin, width=200)

#     bin = bitarray()
#     print("FP32 directly to bin:    ", end=None)
#     bin.frombytes(struct.pack("!f", deltas_fp[idx]))
    
#     util.pprint(bin, width=200)

## Calculate Entropy

In [3]:
total_freq = {}
entropy = 0
for idx, bit_size in enumerate(sorted(dt_freq.keys())):
    chks = chunks[bit_size]
    freq = dt_freq[bit_size]
    total_freq += freq
    for i in freq:
        p_i = freq[i] / sum(freq.values())
        entropy += - p_i * np.log2(p_i)
print("Max comp factor:", entropy)

NameError: name 'dt_freq' is not defined

## Encoding Huffman

In [ ]:
#%pip install dahuffman
from dahuffman import HuffmanCodec


glob_size_huff = 0
glob_size_normal = 0
for idx, bit_size in enumerate(sorted(dt_freq.keys())):
    if bit_size == 0:
        continue
    chks = chunks[bit_size]
    dts = [item for sublist in chks for item in sublist]

    # Make non-sparse freq table
    dt_freqs = dict(dt_freq_sorted[bit_size])
    # dt_freqs_full = {}
    # for i in range(max(dts) + 1):
    #     if i in dt_freqs.keys():
    #         dt_freqs_full[i] = dt_freqs[i] + 1
    #     else:
    #         dt_freqs_full[i] = 1

    #print(dt_freqs_full)
    codec = HuffmanCodec.from_frequencies(dt_freqs)

    tot_size_huff = 0
    tot_size_normal = 0
    for chk in chks:
        encoded = codec.encode(chk)
        USE_TOGGLE_BIT = True
        if USE_TOGGLE_BIT:
            min_size = min(len(encoded) * 8, len(chk) * bit_size)
            tot_size_huff += min_size + 1
        else:
            tot_size_huff += len(encoded) * 8
        #codec.decode(encoded)
        tot_size_normal += len(chk) * bit_size

    print(f"---: {bit_size} : DELTAS {len(dts)} ({len(dts) / len(dt_xs + dt_ys)}) :---")
    print("Huf:", tot_size_huff)
    print("Nor:", tot_size_normal)
    print("Percentage:", tot_size_huff / tot_size_normal)
    glob_size_huff += tot_size_huff
    glob_size_normal += tot_size_normal


print(f"\n---: IN TOTAL :---")
print("Huf:", glob_size_huff)
print("Nor:", glob_size_normal)
print("Percentage:", glob_size_huff / glob_size_normal)


NameError: name 'dt_freq' is not defined

In [ ]:
#%pip install golomb_coding

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for golomb_coding: filename=golomb_coding-1.0.6-py3-none-any.whl size=3532 sha256=2e6794448538b941d73b3e5aaa1f956766861061f6b46df74f07aa43309bcde0
  Stored in directory: /home/simonsigge/.cache/pip/wheels/bc/94/41/10229b1dfc12e83ed9afb144fb573aef7d8623773cd6ad686d
  Created wheel for minimal_binary_coding: filename=minimal_binary_coding-1.0.2-py3-none-any.whl size=3457 sha256=71804889a6b2b488ae662b93928376659332b30bab058f1279a114ef41e30897
  Stored in directory: /home/simonsigge/.cache/pip/wheels/22/91/34/87dd1f6a2fe97604933daa026ba96defd0b5b18d940baba30e
  Created wheel for unary_coding: filename=unary_coding-1.0.5-py3-none-any.whl size=3081 sha256=334a9cc7472f0e2d554009a0eb1b5b3bff2758795b68a10a62021ffb2249916e
  Stored in directory: /home/simonsigge/.cache/pip/wheels/e1/12/c1/73c44284ca0ee1e391985da8686ed8d3f3837d9746f709ca4b
Successfully built g

In [4]:
#%pip install dahuffman
from golomb_coding import golomb_coding


glob_size_huff = 0
glob_size_normal = 0
for idx, bit_size in enumerate(sorted(dt_freq.keys())):
    if bit_size == 0:
        continue
    chks = chunks[bit_size]
    dts = [item for sublist in chks for item in sublist]

    # Make non-sparse freq table
    dt_freqs = dict(dt_freq_sorted[bit_size])
    # dt_freqs_full = {}
    # for i in range(max(dts) + 1):
    #     if i in dt_freqs.keys():
    #         dt_freqs_full[i] = dt_freqs[i] + 1
    #     else:
    #         dt_freqs_full[i] = 1

    #print(dt_freqs_full)
    delta_mean = sum(dts) / len(dts)
    golden_ratio = (math.sqrt(5) + 1) / 2
    param = max(0, 1 + math.floor(math.log2(math.log(golden_ratio - 1) / math.log(delta_mean /(delta_mean + 1)))))

    tot_size_huff = 0
    tot_size_normal = 0
    for chk in chks:
        encoded = [golomb_coding(d, param) for d in chk]
        #print(encoded)
        tot_size_huff += len(encoded)
        #codec.decode(encoded)
        tot_size_normal += len(chk) * bit_size

    print(f"---: {bit_size} : DELTAS {len(dts)} ({len(dts) / len(dt_xs + dt_ys)}) :---")
    print("Huf:", tot_size_huff)
    print("Nor:", tot_size_normal)
    print("Percentage:", tot_size_huff / tot_size_normal)
    glob_size_huff += tot_size_huff
    glob_size_normal += tot_size_normal


print(f"\n---: IN TOTAL :---")
print("Huf:", glob_size_huff)
print("Nor:", glob_size_normal)
print("Percentage:", glob_size_huff / glob_size_normal)


NameError: name 'dt_freq' is not defined